In [16]:
%pip install langchain openai langchain-community langchain-openai html2text playwright beautifulsoup4

Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libavif.so.13                                    ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen.so.0                                   ║
║     libmanette-0.2.so.0                              ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.12/dist-packages/playwright/driver/package/lib/server/registry/dependencies.js:269:9)
    at process.processTicksAndRejections (node:internal/process/task_queues:105

NameError: name 'nest_asyncio' is not defined

In [ ]:
!playwright install

In [17]:
!pip install nest-asyncio
import nest_asyncio

nest_asyncio.apply()

<frozen posixpath>:82: RuntimeWarning: coroutine 'AsyncChromiumLoader.ascrape_playwright' was never awaited


In [18]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [19]:
from langchain_openai import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate

from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import Html2TextTransformer

urls = ["https://docs.nestjs.com/custom-decorators"]
loader = AsyncChromiumLoader(urls, user_agent="MyCrawler/1.0")
html_docs = loader.load()  # 렌더링된 HTML

# HTML → 깔끔한 텍스트 변환
html2text = Html2TextTransformer()
texts = html2text.transform_documents(html_docs)

# 뉴스기사의 본문을 Chunk 단위로 쪼갬
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=3000,     # 쪼개는 글자수
    chunk_overlap=300,   # 오버랩 글자수
    length_function=len,
    is_separator_regex=False,
)

# 웹사이트 내용 크롤링 후 Chunk 단위로 분할
docs = text_splitter.split_documents(texts)
docs

[Document(metadata={'source': 'https://docs.nestjs.com/custom-decorators'}, page_content='dark_mode\n\nSearch```K`\n\n  * Courses\n  * Enterprise\n  * NEW Devtools\n  * Deploy with Mau \n\n### Introduction\n\n### Overview\n\n  * First steps \n  * Controllers \n  * Providers \n  * Modules \n  * Middleware \n  * Exception filters \n  * Pipes \n  * Guards \n  * Interceptors \n  * Custom decorators \n\n### Fundamentals\n\n  * Custom providers \n  * Asynchronous providers \n  * Dynamic modules \n  * Injection scopes \n  * Circular dependency \n  * Module reference \n  * Lazy-loading modules \n  * Execution context \n  * Lifecycle events \n  * Discovery service \n  * Platform agnosticism \n  * Testing \n\n### Techniques\n\n  * Configuration \n  * Database \n  * Mongo \n  * Validation \n  * Caching \n  * Serialization \n  * Versioning \n  * Task scheduling \n  * Queues \n  * Logging \n  * Cookies \n  * Events \n  * Compression \n  * File upload \n  * Streaming files \n  * HTTP module \n  * Se

In [ ]:
# 각 Chunk 단위의 템플릿
template = '''다음의 내용을 한글로 요약해줘:

{text}
'''

# 전체 문서(혹은 전체 Chunk)에 대한 지시(instruct) 정의
combine_template = '''{text}

요약의 결과는 다음의 형식으로 작성해줘:
제목: 웹페이지 제지
주요내용: 한 줄로 요약된 내용
내용: 주요내용을 불렛포인트 형식으로 작성
'''

# 템플릿 생성
prompt = PromptTemplate(template=template, input_variables=['text'])
combine_prompt = PromptTemplate(template=combine_template, input_variables=['text'])
print(prompt)
print(combine_prompt)

input_variables=['text'] input_types={} partial_variables={} template='다음의 내용을 한글로 요약해줘:\n\n{text}\n'
input_variables=['text'] input_types={} partial_variables={} template='{text}\n\n요약의 결과는 다음의 형식으로 작성해줘:\n제목: 신문기사의 제목\n주요내용: 한 줄로 요약된 내용\n작성자: 김철수 대리\n내용: 주요내용을 불렛포인트 형식으로 작성\n'


In [21]:
# LLM 객체 생성
llm = ChatOpenAI(temperature=0,
                 model_name='gpt-5-mini')

In [24]:
# 요약을 도와주는 load_summarize_chain
chain = load_summarize_chain(llm,
                             map_prompt=prompt,
                             combine_prompt=combine_prompt,
                             chain_type='map_reduce',
                             verbose=False)

# 실행결과
output_text = chain.invoke(docs).get('output_text')
print(output_text)

제목: NestJS 문서 요약 — 커스텀 파라미터 데코레이터와 데코레이터 활용법 집중 정리

주요내용: NestJS에서 커스텀 파라미터 데코레이터를 통해 요청의 user 등 특정 데이터를 주입하고, 파이프와 데코레이터 합성으로 깔끔하고 타입 안전한 컨트롤러 코드를 작성하는 방법을 설명.

작성자: 김철수 대리

내용:
- 문서 개요
  - NestJS 기초(컨트롤러·프로바이더·모듈 등)부터 고급 주제(마이크로서비스, GraphQL, WebSocket, 배포 등)를 포괄.
  - Version 10 주요 개선: 커스텀 라우트/파라미터 데코레이터, 데이터 전달, 파이프 활용, 데코레이터 합성 등.

- 데코레이터 기본
  - ES2016 스타일 데코레이터는 데코레이터 표현식을 반환하는 함수이며 클래스·메서드·프로퍼티 등에 @를 붙여 사용.
  - Nest는 여러 기본 파라미터 데코레이터(@Req, @Res, @Param, @Body, @Query, @Headers, @Ip 등)를 제공하며 내부적으로 Express/Fastify 요청 객체의 일부를 매핑.

- 파라미터 데코레이터 매핑(주요 예)
  - @Request(), @Req() → req
  - @Response(), @Res() → res
  - @Next() → next
  - @Session() → req.session
  - @Param(param?) → req.params / req.params[param]
  - @Body(param?) → req.body / req.body[param]
  - @Query(param?) → req.query / req.query[param]
  - @Headers(param?) → req.headers / req.headers[param]
  - @Ip() → req.ip

- 커스텀 파라미터 데코레이터(요지)
  - createParamDecorator와 ExecutionContext를 사용해 재사용 가능한 데코레이터 생성 가능.
  - 보편적 패턴: requ